## What is a neural network?

Neural network is a collection of neurons that are connected by layers. Each neuron is small computing until that performs simple calculations to collectively solve a problem. They are organized in layers. There are 3 types of layers: 
* input layer
* hidden layer
* output layer

Each layer contains a number of neurons, except for the input layer. Neural networks mimic the way a human brain processes information.

### Components of a neural network

#### Activation function
`Activation function determines whether a neuron should be activated or not`. The computations that happen in a neural network include applying an activation function. `If a neuron activates, then it means the input is important`. They are different kinds of activation functions. `The choice of which activation funciton to use dependes on what you want the output to be`. Another important role of an activation function is to `add non-linearity to the model`.
  * *Binary* used to set an output node to 1 if function result is positive and 0 if the function result is negative

$$
f(x)= {\small \begin{cases} 0, & \text{if } x < 0\\ 1, & \text{if } x\geq 0\\ \end{cases}}
$$

  * *Sigmod* is used to predict the probability of an output node being between 0 and 1

$$
f(x) = {\large \frac{1}{1+e^{-x}}}
$$

  * *Tanh* is used to predict the probability of an output node being between -1 and 1. Used in classification use cases.

$$
f(x) = {\large \frac{e^{x} - e^{-x}}{e^{x} + e^{-x}}}
$$

  * *ReLU(Rectified Linear Unit)* used to set the output node to 0 if function result is negative and keeps the result value if the result is a postive values.

$$
f(x)= {\small \begin{cases} 0, & \text{if } x < 0\\ x, & \text{if } x\geq 0\\ \end{cases}}
$$

#### Weights
`Weights` influence how well the output of our network will come close to the expected output value. As an input enter the neuron, it gets multiplied by a weight value and the resulting output is either observed, or passed to the next layer in the neural network. Weights for all neurons in a layer are organize into one tensor.

#### Bias
`Bias` makes up the difference between the activation function's output and its intended output. A low bias suggest that the network is making more assumptions about the form of the output, whereas a high bias value makes less assumptions about the form of the output.


We can say that output y of a neural nework layer with weights W and bias b is computed as summation of the inputs multiply by the weights plus the bias

$$
x = \sum{(weights * inputs) + bias}
$$

where f(x) is the activation function.

## Build a neural network

Neural networks are comprised of layer/modules that perform operations on data. The `torch.nn` namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the `nn.Module`. A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In the following sections, we will build a neural network to classify images in the FashionMNIST dataset.

In [1]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [2]:
%matplotlib inline
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### Get a hardware device for training

In [3]:
# check the paltform, Apple Silicon or Linux
import os, platform

torch_device="cpu"

if 'kaggle' in os.environ.get('KAGGLE_URL_BASE','localhost'):
    torch_device = 'cuda'
else:
    torch_device = 'mps' if platform.system() == 'Darwin' else 'cpu'

In [4]:
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

In [5]:
torch_device

'cuda'

### Define the class

We define our neural network by subclassing `nn.Module`, and initialize the neural network layers in `__init__`. Every `nn.Module` subclass implements the operations on input data in the `forward` method.

Our neural network are composed of the following:

* The input layer with 28x28 or 784 features/pixels
* The first linear module takes the input 784 features and transforms it to a hidden layer with 512 features
* The ReLU activation function will be applied in the transformation
* The second linear module take 512 features as input from the first hidden layer and transforms it to the next hidden layer with 512 features
* The ReLU activation function will be applied in the transformation
* The third linear module take 512 features as input form the second hidden layer and transforms it  to the output layer with 10, which is the number of classes
* The ReLU activation function will be applied in the transformation


In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    def forward(self,x):
        x =self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Move the Neural Network to the device we defined earlier and print the model's architecture.

In [7]:
model = NeuralNetwork().to(torch_device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


To use the model, we pass it the input data. This executes the models `forward`, along with some background operation. However, do not call `model.forward()` directly. Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class. We get the prediction densities by passing it through an instance of the `nn.Softmax`.

In [8]:
X =torch.rand(1,28,28, device=torch_device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([8], device='cuda:0')


### Weight and Bias

The `nn.linear` module randomly initialized the *weights* and *bias* for each layer and internally stores the values in Tensors.

In [9]:
print(f'First Linear weights: {model.linear_relu_stack[0].weight} \n')
print(f'First Linear bias: {model.linear_relu_stack[0].bias} \n')

First Linear weights: Parameter containing:
tensor([[ 0.0161,  0.0330, -0.0175,  ..., -0.0113,  0.0252, -0.0023],
        [ 0.0098, -0.0076, -0.0188,  ..., -0.0137, -0.0007, -0.0024],
        [-0.0281, -0.0046, -0.0146,  ..., -0.0147,  0.0268, -0.0234],
        ...,
        [-0.0244,  0.0308,  0.0331,  ...,  0.0077,  0.0046, -0.0251],
        [ 0.0284,  0.0164, -0.0332,  ..., -0.0063,  0.0288,  0.0100],
        [ 0.0259, -0.0089,  0.0291,  ..., -0.0038,  0.0060, -0.0152]],
       device='cuda:0', requires_grad=True) 

First Linear bias: Parameter containing:
tensor([-8.4437e-03,  1.4467e-02,  4.9862e-03,  2.7656e-02,  1.9958e-03,
        -1.3240e-02, -2.0032e-03,  7.3737e-03,  1.7689e-02,  2.2507e-02,
         7.3073e-03,  2.0721e-02, -2.4472e-02, -1.9952e-02,  3.0265e-02,
        -1.1435e-02,  3.1261e-02,  2.5780e-02, -2.3674e-03, -7.2519e-03,
        -2.0901e-03, -7.8312e-03,  1.1703e-02,  9.3221e-03,  1.5358e-02,
        -1.7135e-02,  9.2835e-04,  2.8485e-02,  1.1371e-02, -1.0883e-0

### Model layers

Let's break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [10]:
input_sample = torch.rand(3,28,28)
print(input_sample.size())

torch.Size([3, 28, 28])


### nn.Flatten

We initialize the `nn.Flatten` layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (the minibatch dimension (at dim=0) is maintained). Each of the pixels are pass to the input layer of the neural network.

In [11]:
flatten = nn.Flatten()
flat_image = flatten(input_sample)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear

The linear layer is a module that applies a linear transformation on the input using it's stored weights and biases. The gayscale value of each pixel in the input layer will be connected to neurons in the hidden layer for calculation. The calculation used for the transformation is `weight` * `input` + `bias`.

In [12]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

Non-linear activations are what create the complex mappings between the model's input and output. They are applied after linear tranformations to introduce `nonlinearity`, helping neural networks learn a wide variety of phenomena. In this model, we use `nn.ReLU` between our linear layers, but there's other activations to introduce non-linearity in your model.

The ReLU activation function takes the output from the linear layer calculation and replaces the negative values with zeros.

Linear output:
$$
{ x = {weight * input + bias}}
$$
ReLU:

$$
f(x)= {\small \begin{cases} 0, & \text{if } x < 0\\ x, & \text{if } x\geq 0\\ \end{cases}}
$$

In [13]:
print(f'Before ReLU: {hidden1}\n\n')
hidden1 = nn.ReLU()(hidden1)
print(f'After ReLU: {hidden1}')

Before ReLU: tensor([[-0.1619, -0.8146,  0.0379, -0.0796, -0.0049, -0.6402, -0.2999,  0.2494,
         -0.0133,  0.4261, -0.0852,  0.0131, -0.0867,  0.6185,  0.1049, -0.0262,
          0.3902,  0.3320, -0.2574, -0.5516],
        [-0.1508, -0.5107, -0.1021, -0.2224, -0.1385, -0.2996, -0.2088, -0.0274,
         -0.2109,  0.6311, -0.1400, -0.4624, -0.1137,  0.5212,  0.1354,  0.1216,
          0.8446,  0.5370, -0.4295, -0.5016],
        [-0.0712, -0.6318, -0.3181, -0.0929, -0.0306, -0.2385,  0.0410, -0.3792,
         -0.1304,  0.6893, -0.1858, -0.2376, -0.1521,  0.0300,  0.0387,  0.3418,
          0.6033,  0.5135, -0.1706, -0.6416]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0379, 0.0000, 0.0000, 0.0000, 0.0000, 0.2494, 0.0000,
         0.4261, 0.0000, 0.0131, 0.0000, 0.6185, 0.1049, 0.0000, 0.3902, 0.3320,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.6311, 0.0000, 0.0000, 0.0000, 0.5212, 0.13

### nn.Sequential

`nn.Sequential` is an ordered container of modules. The data is passed through all the modulers in the same order as defined. You can use sequential containers to put together a quick network like `seq_modules`.

In [14]:
seq_modules = nn.Sequential(
    flatten,
    layer1, nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### nn.Softmax

The last linear layer of the neural network returns `logits` - raw values in [-infty, infty], which are passed to the `nn.Softmax` module. The Softmax activation function is used to calcualte the probability of the output from the neural network. It is only used on output layer  of a neural network. the results are scaled to values [0,1] representing the model's predicted densities for each class. `dim` parameter indicates the dimension along which the result values must sum to 1. The node with the highest probability predicts the desired output.

In [15]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

### Model parameters

Many layers inside a neural network are `parameterized`, i.e. have associated weights and biases that are optimized during training. Subclassing `nn.Module` automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model's `parameters()` or `named_parameters()` methods.

In [16]:
print('Model structure:', model, '\n\n')

for name, param in model.named_parameters():
    print(f'Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n')

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0161,  0.0330, -0.0175,  ..., -0.0113,  0.0252, -0.0023],
        [ 0.0098, -0.0076, -0.0188,  ..., -0.0137, -0.0007, -0.0024]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0084,  0.0145], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0043,  0.0079,  0.0286,  ...,  0.0199,  0.0015, -0.0135],
        [ 0.0030, -0.0256,  0.0300,  ...,  0.0230, -0.0049, -0.0169]],
       device='cuda:0